# Sony Video Camera Data: Segmentation, Image Extraction, and Preparation

In [4]:
## Required python libraries
import pandas as pd
import numpy as np
import scipy as sp
import scipy.signal
import os
import math
import sys
import shutil
from scipy.interpolate import griddata
import subprocess
# to make this notebook's output stable across runs
np.random.seed(42)

pd.plotting.register_matplotlib_converters()


import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
%matplotlib inline
import seaborn as sns
import librosa
import matplotlib.font_manager as font_manager

#---------------opencv------------------------
import cv2
from IPython.display import Image, display
from tqdm.notebook import tqdm

In [5]:
from moviepy.editor import VideoFileClip
import imageio_ffmpeg as ffmpeg

In [6]:
# Where to save the figures, and dataset locations
PROJECT_ROOT_DIR = "../"

Multimodal_dataset_PATH = "/home/chenlequn/pan1/Dataset/LDED_acoustic_visual_monitoring_dataset"

Video_path = os.path.join(Multimodal_dataset_PATH, f'Sony Camera')
IMAGE_PATH = os.path.join(PROJECT_ROOT_DIR, "result_images", 'Sony-Camera-Image')

os.makedirs(IMAGE_PATH, exist_ok=True)

## function for automatically save the diagram/graph into the folder 
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGE_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [7]:
# Define function to save figures
def save_frames(frame, sample_index, counter, image_output_folder, fig_extension="png"):
    filename = f'sample_{sample_index}_{counter}.{fig_extension}'
    path = os.path.join(image_output_folder, filename)
    # print("Saving frame", filename)
    cv2.imwrite(path, frame)
    # cv2.imwrite(path, frame, [int(cv2.IMWRITE_JPEG_QUALITY), 100])


# Function to display video information
def display_video_info(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_duration = total_frames / fps
    # Convert total duration to minutes and seconds
    total_duration_min = int(total_duration // 60)
    total_duration_sec = int(total_duration % 60)

    print(f"Video FPS: {fps}")
    print(f"Total Frames: {total_frames}")
    print(f"Total Duration (seconds): {total_duration}")
    print(f"Total Duration: {total_duration_min} min {total_duration_sec} seconds")


    cap.release()

# Function to crop video by time and save frames
def crop_video_and_save_frames(video_path, image_output_folder, start_time, end_time, sample_index, target_fps=25):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    frame_interval = int(fps / target_fps)
    
    # Calculate total frames to process and initialize tqdm
    total_frames_to_process = (end_frame - start_frame) // frame_interval
    pbar = tqdm(total=total_frames_to_process, desc="Processing frames")
    
    counter = 1  # Initialize frame counter
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
    while True:
        ret, frame = cap.read()
        if not ret or cap.get(cv2.CAP_PROP_POS_FRAMES) > end_frame:
            break  # Stop if the end of the segment is reached or on read error
        
        if (counter - 1) % frame_interval == 0:
            save_frames(frame, sample_index, counter, image_output_folder)
            pbar.update(1)
        
        counter += 1

    pbar.close()
    cap.release()


def format_time(seconds):
    """Converts time in seconds to HH:MM:SS format."""
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

def crop_video_and_save_frames_ffmpeg(video_path, image_output_folder, start_time, end_time, sample_index, target_fps=25):
    # Convert start_time and end_time to HH:MM:SS format
    start_timestamp = format_time(start_time)
    duration = end_time - start_time
    duration_timestamp = format_time(duration)
    total_frames = int(duration * target_fps)

    # Ensure output folder exists
    if not os.path.exists(image_output_folder):
        os.makedirs(image_output_folder)

    # Output pattern for frames
    output_pattern = os.path.join(image_output_folder, f"sample_{sample_index}_%d.png")

    # Build the FFmpeg command
    command = [
        'ffmpeg',
        '-ss', start_timestamp,                 # Start time
        '-t', duration_timestamp,               # Duration to process
        '-i', video_path,                       # Input file path
        '-vf', f'yadif,fps={target_fps}',       # Video filters
        '-q:v', '1',                            # Output quality (lower is better)
        '-start_number', '1',                   # Start numbering frames at 0
        '-progress', 'pipe:1',                  # Output progress to pipe
        output_pattern
    ]

    # Start the FFmpeg process and include a progress bar
    subprocess.run(command, check=True)

## Segment MTS video file into Images

In [51]:
sample_index = 24

# ----------------Define input video file path---------------
raw_video_file = os.path.join(Video_path, f'{sample_index}.MTS') 

# Output folders
image_output_folder = os.path.join(Multimodal_dataset_PATH, '25Hz',str(sample_index), 'sony-images')
sony_camera_image_folder = os.path.join(Multimodal_dataset_PATH, '25Hz',str(sample_index), 'sony_camera_images')

# Create output folders if they don't exist
if not os.path.exists(image_output_folder):
    os.makedirs(image_output_folder)

# Create output folders if they don't exist
if not os.path.exists(sony_camera_image_folder):
    os.makedirs(sony_camera_image_folder)

# Display video information
display_video_info(raw_video_file)

Video FPS: 25.0
Total Frames: 21086
Total Duration (seconds): 843.44
Total Duration: 14 min 3 seconds


In [18]:
# Define start and end times for cropping
# Sample 21: 0 - 182 seconds, in total, 4419 images (initial several layers are missing in the sony videos)
# Sample 22: 0 - 179 seconds, in total 
# Sample 23: 0 - 6 min 47 seconds = 407
# Sample 24: 0 - 13 min 27 seconds = 807
# Sample 26: 0 - 6 min 43 seconds = 403

start_time = 0  # Start time in seconds 
end_time = 807   # End time in seconds

# Crop the video and save frames at 25 Hz
# crop_video_and_save_frames(raw_video_file, image_output_folder, start_time, end_time, sample_index, target_fps=25)

crop_video_and_save_frames_ffmpeg(raw_video_file, image_output_folder, start_time, end_time, sample_index, target_fps=25)

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


frame=73
fps=0.00
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=1640000
out_time_ms=1640000
out_time=00:00:01.640000
dup_frames=0
drop_frames=0
speed=3.26x
progress=continue


frame=147
fps=145.59
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=4600000
out_time_ms=4600000
out_time=00:00:04.600000
dup_frames=0
drop_frames=0
speed=4.56x
progress=continue


frame=216
fps=142.33
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=7360000
out_time_ms=7360000
out_time=00:00:07.360000
dup_frames=0
drop_frames=0
speed=4.85x
progress=continue


frame=288
fps=142.33
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=10240000
out_time_ms=10240000
out_time=00:00:10.240000
dup_frames=0
drop_frames=0
speed=5.06x
progress=continue


...

frame=20094
fps=53.52
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=802480000
out_time_ms=802480000
out_time=00:13:22.480000
dup_frames=0
drop_frames=0
speed=2.14x
progress=continue


frame=20113
fps=53.50
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=803240000
out_time_ms=803240000
out_time=00:13:23.240000
dup_frames=0
drop_frames=0
speed=2.14x
progress=continue


frame=20127
fps=53.44
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=803800000
out_time_ms=803800000
out_time=00:13:23.800000
dup_frames=0
drop_frames=0
speed=2.13x
progress=continue
frame=20162
fps=53.46
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=805200000
out_time_ms=805200000
out_time=00:13:25.200000
dup_frames=0
drop_frames=0
speed=2.14x
progress=continue


frame=20175
fps=53.34
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=807000000
out_time_ms=807000000
out_time=00:13:27.000000
dup_frames=0
drop_frames=0
speed=2.13x
progress=end


frame=20175 fps= 53 q=-0.0 Lsize=N/A time=00:13:27.00 bitrate=N/A speed=2.13x    
video:36956096kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown


## Fix the index to match with the coaxial image stream.

In [52]:
import os

# Calculate the index offset
index_offset = 1-32
# Sample 21: offset is 5030 (coaxial) - 4430
# Sample 22: offset is 4623 (coaxial) - 4469
# Sample 23: offset is 11155 (coaxial) - 10104
# Sample 24: offset is 1 - 32 (special case, the later part of 24 is disrgarded)
# Sample 26: offset is 10749 (coaxial) - 10010

# Get all PNG files in the directory
image_files = [f for f in os.listdir(image_output_folder) if f.endswith(".png")]

In [53]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def copy_and_rename_file(filename, image_output_folder, sony_camera_image_folder, sample_index, index_offset):
    parts = filename.split('_')
    if len(parts) == 3:
        current_index_part = parts[2].split('.')[0]  # This gets the numeric part of '<current_index>.png'
        try:
            current_index = int(current_index_part)
            new_index = current_index + index_offset
            new_filename = f'sample_{sample_index}_{new_index}.png'
            current_file_path = os.path.join(image_output_folder, filename)
            new_file_path = os.path.join(sony_camera_image_folder, new_filename)
            shutil.copy2(current_file_path, new_file_path)
            return filename, True
        except ValueError:
            return filename, False
    return filename, False

# Assume image_files, image_output_folder, sony_camera_image_folder, sample_index, index_offset are defined
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    future_to_file = {executor.submit(copy_and_rename_file, filename, image_output_folder, sony_camera_image_folder, sample_index, index_offset): filename for filename in image_files}
    
    # Initialize progress bar
    progress = tqdm(as_completed(future_to_file), total=len(image_files), desc="Renaming images")
    for future in progress:
        filename, success = future.result()

Renaming images:   0%|          | 0/20175 [00:00<?, ?it/s]

## Further manual adjustment

In [54]:
import re
def renumber_images_sequentially(folder_path):
    """
    Renumber image files in a folder to ensure sequential numbering for each sample index.

    Parameters:
    - folder_path: The path to the folder containing the images.
    """
    # Regex to match and extract sample_index and index from filenames
    pattern = re.compile(r"sample_(\d+)_(\d+).png")
    
    # Dictionary to hold file paths grouped by sample_index
    files_dict = {}

    # Populate files_dict with filenames, grouped by sample_index
    for filename in os.listdir(folder_path):
        match = pattern.match(filename)
        if match:
            sample_index, index = match.groups()
            if sample_index not in files_dict:
                files_dict[sample_index] = []
            files_dict[sample_index].append((int(index), filename))
    
    # Iterate through each group of files and renumber them
    for sample_index, files in files_dict.items():
        # Sort files by their original index
        files.sort()
        for new_index, (_, filename) in enumerate(tqdm(files, desc=f"Renaming sample {sample_index}"), start=1):
            new_filename = f"sample_{sample_index}_{new_index}.png"
            os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))


In [68]:
trial = "/home/chenlequn/pan1/Dataset/LDED_acoustic_visual_monitoring_dataset/25Hz/24/trial"
renumber_images_sequentially(sony_camera_image_folder)

Renaming sample 24:   0%|          | 0/16325 [00:00<?, ?it/s]